# Perfil de comércio

## Setup

### Criar directorias de apoio 

In [1]:
from pathlib import Path

Path('support').mkdir(parents=True, exist_ok=True)
Path('reports').mkdir(parents=True, exist_ok=True)

### Criar ficheiro de configuração se não existir

O ficheiro de configuração serve para armazenar
a APIKEY que dá acesso a mais de 500 linhas de
resultados.


In [2]:
import os

fname = 'config.ini'
content = """
# Ficheiro de configuração
[comtrade]
# Add API Key. DO NOT SHARE
key = 
"""
if not os.path.isfile(fname):
  print("Criando ficheiro de configuração")
  with open(fname,'w') as f:
    f.write(content)


### Obter uma chave de acesso à API

Para aceder à UN Comtrade via API sem limites é necessário uma chave de acesso,
de outro modo os resultados são limitados a 500 linhas.

Para obter a chave de acsso:
* Registo em https://comtradedeveloper.un.org/
* Ir para _Products_ 
* Selecionar "Premium Individual APIs" (https://comtradedeveloper.un.org/product#product=dataapis)
* Escolher _Subscribe to "comtrade - v1"_  
* Esperar pelo email com a chave da API key (demora alguns dias)
* Copiar a chave para o local indicado no ficheiro `config.ini` antes
  de executar o resto do notebook.


### Inicializar a API e transferir os códigos usados

In [3]:
import os
import configparser
import comtrade

if os.path.isfile(fname):

    config = configparser.ConfigParser()
    config.read('config.ini')
    # get API Key or set to None    
    APIKEY = config['comtrade'].get('key', None)

comtrade.init(APIKEY, force_init=True)
comtrade.CALLS_PER_PERIOD=30  # Period default is 60 seconds


contrade.py initialized


## Country and period of interest

Choose country and years of interest

In [4]:
country_of_interest = comtrade.m49_angola
country_of_interest_desc = comtrade.COUNTRY_CODES[country_of_interest]

years_of_interest='2007,2008,2009,2010,2014,2015,2016,2017,2018,2019,2020,2021'
print(country_of_interest_desc, years_of_interest)

Angola 2007,2008,2009,2010,2014,2015,2016,2017,2018,2019,2020,2021


## Trade balance

In [5]:
import pandas as pd

pd.options.display.float_format = '{:,.2f}'.format


global_trade = comtrade.get_global_stats(country_of_interest, period=years_of_interest,
                                         symmetric_values=True)


In [6]:
global_trade

flowCode,M,M<X,X,X<M,trade_balance,trade_volume,trade_balance （X<M-M),trade_volume （X<M-M)
period,,,,,,,,
2007,"11,094,843,413.00","12,662,439,392.45","44,177,783,071.00","41,711,887,538.55","33,082,939,658.00","55,272,626,484.00","30,617,044,125.55","52,806,730,951.55"
2008,0.00,"20,453,079,573.81",0.00,"67,748,979,650.35",0.00,0.00,"67,748,979,650.35","67,748,979,650.35"
2009,"23,918,854,361.00","17,838,411,060.41","40,639,411,732.00","39,936,869,385.44","16,720,557,371.00","64,558,266,093.00","16,018,015,024.44","63,855,723,746.44"
2010,"18,143,268,908.00","15,871,622,642.40","52,612,114,761.00","53,459,185,324.68","34,468,845,853.00","70,755,383,669.00","35,315,916,416.68","71,602,454,232.68"
2014,"28,753,499,329.00","28,333,262,583.05","58,672,369,190.00","64,694,135,949.55","29,918,869,861.00","87,425,868,519.00","35,940,636,620.55","93,447,635,278.55"
2015,"21,549,256,530.51","18,429,407,926.75","33,924,937,478.31","36,829,899,264.38","12,375,680,947.80","55,474,194,008.82","15,280,642,733.87","58,379,155,794.89"
2016,"14,347,710,497.80","11,567,121,254.47","28,057,499,522.39","28,304,363,119.09","13,709,789,024.59","42,405,210,020.19","13,956,652,621.29","42,652,073,616.89"
2017,"15,462,313,478.23","11,855,347,823.95","34,904,881,109.37","36,704,443,851.05","19,442,567,631.14","50,367,194,587.60","21,242,130,372.82","52,166,757,329.28"
2018,"16,006,910,579.06","10,328,693,528.99","40,664,718,814.96","44,183,954,089.23","24,657,808,235.90","56,671,629,394.02","28,177,043,510.17","60,190,864,668.29"


## Exports

Usamos as importações declaradas pelos parceiros para obter as
exportações do país de interesse.

### Main export partners of the country

In [7]:

# We get the exports from the reported imports of all the countries
# i.e. instead of asking Angola for export data we ask the world for import data from Angola

# years_of_interest = comtrade.year_range(2020, 2021)  
df = comtrade.get_data(reporterCode=None,
                       typeCode='C',  # commodities
                       freqCode='A',  # annual
                       partnerCode=country_of_interest,
                       period=years_of_interest,
                       motCode=0,  # all means of transport
                       flowCode='M',
                       timeout=120)

Utility function for formatting columns in the trade profile dataframes
"""

In [8]:
def make_format(cols:list):
    f = {col:'{0:.3%}' for col in cols if col.endswith('_perc')}
    f.update({col:'${0:,.0f}' for col in cols if col.endswith('_sum')})
    f.update({'primaryValue':'${0:,.0f}'})
    return f


In [9]:
import pandas as pd
# show 500 lines
pd.options.display.max_rows=500
pd.options.display.float_format='{:,.2f}'.format

# aggregate by year, flow, reporter 
exports = comtrade.total_rank_perc(df,groupby=['refYear','flowCode','reporterDesc'],
                                   col='primaryValue',
                                   prefix='partner',
                                   drop_duplicates=True)

show_only_top = 5 
cols=['refYear','flowCode','reporterDesc','cmdCode','partner_upper_sum','partner_sum','partner_rank','partner_perc','partner_upper_perc']

formats =  make_format(cols)

top_export_partners = exports[exports['partner_rank']<=show_only_top]['reporterDesc'].unique()
top_export_partners_codes = exports[exports['partner_rank']<=show_only_top]['reporterCode'].unique()

print(f"Top {show_only_top} export partners of {country_of_interest_desc} in {years_of_interest}: \n{top_export_partners}")
exports[exports['partner_rank']<=show_only_top][cols].sort_values(['refYear','flowCode','partner_rank','reporterDesc','cmdCode']).style.format(formats)

Top 5 export partners of Angola in 2007,2008,2009,2010,2014,2015,2016,2017,2018,2019,2020,2021: 
['China' 'France and Monaco' 'Asia, not elsewhere specified'
 'South Africa' 'United States of America' 'Canada' 'India' 'Spain'
 'United Arab Emirates' 'Portugal' 'Thailand']


,refYear,flowCode,reporterDesc,cmdCode,partner_upper_sum,partner_sum,partner_rank,partner_perc,partner_upper_perc
147,2007,M,United States of America,TOTAL,"$41,711,887,539","$12,925,965,219",1,30.989%,30.989%
18,2007,M,China,TOTAL,"$41,711,887,539","$12,888,664,603",2,30.899%,30.899%
38,2007,M,France and Monaco,TOTAL,"$41,711,887,539","$2,370,503,388",3,5.683%,5.683%
73,2007,M,"Asia, not elsewhere specified",TOTAL,"$41,711,887,539","$2,121,461,485",4,5.086%,5.086%
101,2007,M,South Africa,TOTAL,"$41,711,887,539","$1,645,793,847",5,3.946%,3.946%
190,2008,M,China,TOTAL,"$67,748,979,650","$22,382,523,829",1,33.037%,33.037%
352,2008,M,United States of America,TOTAL,"$67,748,979,650","$19,497,727,584",2,28.779%,28.779%
211,2008,M,France and Monaco,TOTAL,"$67,748,979,650","$4,010,271,285",3,5.919%,5.919%
305,2008,M,South Africa,TOTAL,"$67,748,979,650","$2,686,472,634",4,3.965%,3.965%
187,2008,M,Canada,TOTAL,"$67,748,979,650","$2,607,324,781",5,3.849%,3.849%


Passamos os países para colunas, mantendo a percentagem e o valor em separado.

In [143]:
pivoted = df.pivot_table(index=['refYear','flowDesc'], columns='reporterDesc', values=['primaryValue','partner_perc'])
pivoted.swaplevel(axis=1).fillna('',inplace=True)
pivoted.sort_index(axis=1, level=0, inplace=True)
pivoted.fillna(0)

partner_perc                                                 \
reporterDesc          Albania Angola Argentina Asia, not elsewhere specified   
refYear flowDesc                                                               
2020    Import           0.00   0.00      0.00                          0.02   
2021    Import           0.00   0.00      0.00                          0.01   

                                                                       ...  \
reporterDesc     Australia Austria Azerbaijan Bahamas Bahrain Belarus  ...   
refYear flowDesc                                                       ...   
2020    Import        0.00    0.00       0.00    0.00    0.00    0.00  ...   
2021    Import        0.00    0.00       0.00    0.00    0.00    0.00  ...   

                  primaryValue                                          \
reporterDesc            Turkey   Uganda   Ukraine United Arab Emirates   
refYear flowDesc                                                         
2020    Import    4,228,350.00 2,766.00 35,298.67     1,018,255,600.93   
2021    Import   10,607,228.00     0.00  1,389.42     1,685,701,656.52   

                                                                         \
reporterDesc     United Kingdom United States of America        Uruguay   
refYear flowDesc                                                          
2020    Import     1,979,755.60           497,345,885.00 213,330,946.83   
2021    Import     7,990,634.18         1,089,250,402.00 292,943,215.91   

                                                    
reporterDesc           Vietnam     Zambia Zimbabwe  
refYear flowDesc                                    
2020    Import    9,066,304.05 211,208.91     0.00  
2021    Import   37,924,016.16 396,117.44   240.94  

[2 rows x 248 columns]

In [144]:
show_only_columns = [(l1, country) for l1,country in pivoted.columns.tolist() if country in top_export_partners]
pivoted[show_only_columns]

partner_perc                                            \
reporterDesc            China India Spain Thailand United Arab Emirates   
refYear flowDesc                                                          
2020    Import           0.61  0.08  0.03     0.05                 0.04   
2021    Import           0.62  0.07  0.02     0.05                 0.05   

                                               primaryValue                   \
reporterDesc     United States of America             China            India   
refYear flowDesc                                                               
2020    Import                       0.02 14,757,730,338.00 2,022,432,688.74   
2021    Import                       0.03 20,908,231,317.00 2,315,947,568.41   

                                                                       \
reporterDesc              Spain         Thailand United Arab Emirates   
refYear flowDesc                                                        
2020    Import   626,580,515.28 1,258,209,112.04     1,018,255,600.93   
2021    Import   739,484,782.77 1,607,756,231.51     1,685,701,656.52   

                                           
reporterDesc     United States of America  
refYear flowDesc                           
2020    Import             497,345,885.00  
2021    Import           1,089,250,402.00

#### Export to Excel

In [11]:
exports.to_excel(f'{country_of_interest_desc}_top_partners.xlsx')
pivoted.to_excel(f'{country_of_interest_desc}_top_partners_cols.xlsx')
pivoted[show_only_columns].to_excel(f'{country_of_interest_desc}_export_top_{show_only_top}_partners_cols.xlsx')

## Exports: products

### Main exported products

In [16]:
import pandas as pd

# show 500 lines of dataframes
pd.set_option('display.max_rows', 500)

show_only_top=5
# years_of_interest = comtrade.year_range(2020, 2021)  


Get the products imported from Angola

In [17]:
df = comtrade.get_data(reporterCode=None,
                       typeCode='C',  # commodities
                       freqCode='A',  # annual
                       partnerCode=country_of_interest,
                       period=years_of_interest,
                       cmdCode='AG4',  # Top level product types
                       motCode=0,  # all means of transport
                       flowCode='M',
                       timeout=120)

/Users/jrc/develop/cipf-comtrade/comtrade.py:395: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  warnings.warn("Query returned different isAggregate values, check for duplicate results when aggregating")


In [18]:
import pandas as pd
# show 500 lines
pd.options.display.max_rows=500
pd.options.display.float_format='{:,.2f}'.format

show_only_top = 5 
exports = comtrade.total_rank_perc(df,['refYear','flowCode','cmdCode'],'primaryValue','product',drop_duplicates=True)
exports = exports[exports['product_rank']<=show_only_top]

cols=['refYear','flowCode','cmdCode','product_sum','product_rank','product_perc','product_upper_sum','product_upper_perc']
formats =  make_format(cols)

top_export_products_codes = exports['cmdCode'].unique()

print(f"Top products exported by {country_of_interest_desc} in {years_of_interest}:\n {top_export_products_codes}")
for cmdCode in top_export_products_codes:
    print(f"   {cmdCode}: {comtrade.HS_CODES.get(cmdCode,cmdCode)}")
exports[cols].sort_values(['refYear','flowCode','product_rank']).style.format(formats)


Top products exported by Angola in 2007,2008,2009,2010,2014,2015,2016,2017,2018,2019,2020,2021:
 ['2709' '2710' '7102' '9999' '8905' '2711' '8901' '7204' '8419' '0303'
 '2516' '2715']
   2709: Petroleum oils and oils obtained from bituminous minerals; crude
   2710: Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of the preparations; waste oils
   7102: Diamonds, whether or not worked, but not mounted or set
   9999: Commodities not specified according to kind
   8905: Light-vessels, fire-floats, dredgers, floating cranes, other vessels; the navigability of which is subsidiary to main function; floating docks, floating, submersible drilling, production platforms
   2711: Petroleum gases and other gaseous hydrocarbons
   8901: Cruise ships, excursion boats, ferry-boats, cargo ships, barges and similar vessels for the transport of person

,refYear,flowCode,cmdCode,product_sum,product_rank,product_perc,product_upper_sum,product_upper_perc
7,2007,M,2709,"$38,353,255,496",1,2.043%,"$41,704,795,548",91.964%
147,2007,M,9999,"$756,136,900",2,0.000%,"$41,704,795,548",1.813%
317,2007,M,8905,"$685,462,362",3,1.644%,"$41,704,795,548",1.644%
54,2007,M,7102,"$629,792,010",4,0.598%,"$41,704,795,548",1.510%
9,2007,M,2710,"$424,218,083",5,0.041%,"$41,704,795,548",1.017%
2118,2008,M,2709,"$65,415,688,359",1,3.246%,"$67,748,939,616",96.556%
2180,2008,M,7102,"$891,139,665",2,0.238%,"$67,748,939,616",1.315%
2120,2008,M,2710,"$503,734,811",3,0.046%,"$67,748,939,616",0.744%
2121,2008,M,2711,"$377,344,746",4,0.094%,"$67,748,939,616",0.557%
2459,2008,M,8901,"$246,575,238",5,0.355%,"$67,748,939,616",0.364%


#### Export to Excel

In [19]:
exports.to_excel(f'{country_of_interest_desc}_exports_products_{exports["refYear"].min()}-{exports["refYear"].max()}.xlsx')

## Export partners / products analysis

What are the partners buying the top exports of country of interest?


In [29]:
import pandas as pd
# show 500 lines
pd.options.display.max_rows=500
pd.options.display.float_format='{:,.2f}'.format

show_only_top = 5 
print(f"Top products exported by {country_of_interest_desc} in {years_of_interest}:\n {top_export_products_codes}")
for cmdCode in top_export_products_codes:
    print(f"   {cmdCode}: {comtrade.HS_CODES.get(cmdCode,cmdCode)}")

print(f"Top export partners ({len(top_export_partners)}): {top_export_partners}")
prefix='cmd_partner'
products_and_partners = comtrade.total_rank_perc(df,['refYear','flowCode','cmdCode','reporterCode'],'primaryValue',prefix,drop_duplicates=False)

cols=['refYear','flowCode','cmdDesc','cmdCode','reporterDesc',f'{prefix}_sum',f'{prefix}_rank',f'{prefix}_upper_sum',f'{prefix}_perc']
formats=make_format(cols)
products_and_partners=products_and_partners[(products_and_partners['cmdCode'].isin(top_export_products_codes) 
                       & (products_and_partners[f'{prefix}_rank']<=show_only_top))]


Top products exported by Angola in 2007,2008,2009,2010,2014,2015,2016,2017,2018,2019,2020,2021:
 ['2709' '2710' '7102' '9999' '8905' '2711' '8901' '7204' '8419' '0303'
 '2516' '2715']
   2709: Petroleum oils and oils obtained from bituminous minerals; crude
   2710: Petroleum oils and oils from bituminous minerals, not crude; preparations n.e.c, containing by weight 70% or more of petroleum oils or oils from bituminous minerals; these being the basic constituents of the preparations; waste oils
   7102: Diamonds, whether or not worked, but not mounted or set
   9999: Commodities not specified according to kind
   8905: Light-vessels, fire-floats, dredgers, floating cranes, other vessels; the navigability of which is subsidiary to main function; floating docks, floating, submersible drilling, production platforms
   2711: Petroleum gases and other gaseous hydrocarbons
   8901: Cruise ships, excursion boats, ferry-boats, cargo ships, barges and similar vessels for the transport of person

### Partners for main export products

In [30]:

products_and_partners[cols].sort_values(['refYear','flowCode',f'{prefix}_upper_sum',f'{prefix}_rank'],
                                         ascending=[True,True,False,True]).style.format(formats) #.hide(axis='index')

,refYear,flowCode,cmdDesc,cmdCode,reporterDesc,cmd_partner_sum,cmd_partner_rank,cmd_partner_upper_sum,cmd_partner_perc
156,2007,M,Petroleum oils and oils obtained from bituminous minerals; crude,2709,China,"$12,879,534,443",1,"$38,353,255,496",33.581%
1900,2007,M,Petroleum oils and oils obtained from bituminous minerals; crude,2709,United States of America,"$12,525,658,693",2,"$38,353,255,496",32.659%
356,2007,M,Petroleum oils and oils obtained from bituminous minerals; crude,2709,France and Monaco,"$2,343,536,013",3,"$38,353,255,496",6.110%
784,2007,M,Petroleum oils and oils obtained from bituminous minerals; crude,2709,"Asia, not elsewhere specified","$2,120,882,527",4,"$38,353,255,496",5.530%
1404,2007,M,Petroleum oils and oils obtained from bituminous minerals; crude,2709,South Africa,"$1,626,436,334",5,"$38,353,255,496",4.241%
1332,2007,M,Commodities not specified according to kind,9999,Netherlands,"$750,598,390",1,"$756,136,900",99.268%
2093,2007,M,Commodities not specified according to kind,9999,United States of America,"$2,707,737",2,"$756,136,900",0.358%
340,2007,M,Commodities not specified according to kind,9999,Denmark,"$1,521,286",3,"$756,136,900",0.201%
2092,2007,M,Commodities not specified according to kind,9999,United Kingdom,"$740,048",4,"$756,136,900",0.098%
733,2007,M,Commodities not specified according to kind,9999,Israel,"$238,000",5,"$756,136,900",0.031%


### Main partners and products

In [36]:
print(f"Top export partners ({len(top_export_partners)}): {top_export_partners}")
prefix='partner_cmd'
partners_and_products = comtrade.total_rank_perc(df,['refYear','flowCode','reporterCode','cmdCode'],'primaryValue',prefix,drop_duplicates=False)

cols=['reporterDesc','refYear','flowCode','cmdDesc','cmdCode',f'{prefix}_sum',f'{prefix}_rank',f'{prefix}_upper_sum',f'{prefix}_perc']
formats=make_format(cols)
partners_and_products=partners_and_products[(partners_and_products['cmdCode'].isin(top_export_products_codes) 
                       & (partners_and_products[f'{prefix}_rank']<=show_only_top))]


partners_and_products[partners_and_products[f'{prefix}_rank']==1][cols].sort_values(['reporterDesc','refYear','flowCode',f'{prefix}_sum'],
                                         ascending=[True,True,True,False]).style.format(formats).hide(axis='index')

Top export partners (11): ['China' 'France and Monaco' 'Asia, not elsewhere specified'
 'South Africa' 'United States of America' 'Canada' 'India' 'Spain'
 'United Arab Emirates' 'Portugal' 'Thailand']


reporterDesc,refYear,flowCode,cmdDesc,cmdCode,partner_cmd_sum,partner_cmd_rank,partner_cmd_upper_sum,partner_cmd_perc
Albania,2019,M,Commodities not specified according to kind,9999,"$53,390",1,"$53,390",100.000%
Albania,2021,M,Commodities not specified according to kind,9999,"$31,371",1,"$31,371",100.000%
Algeria,2007,M,"Fish; frozen, excluding fish fillets and other fish meat of heading 0304",0303,"$49,269",1,"$112,234",43.898%
Algeria,2014,M,"Fish; frozen, excluding fish fillets and other fish meat of heading 0304",0303,"$39,955",1,"$123,167",32.440%
Algeria,2016,M,"Fish; frozen, excluding fish fillets and other fish meat of heading 0304",0303,"$353,002",1,"$637,418",55.380%
Anguilla,2008,M,Commodities not specified according to kind,9999,"$2,894",1,"$7,776",37.217%
Argentina,2010,M,Petroleum oils and oils obtained from bituminous minerals; crude,2709,"$2,316",1,"$4,081",56.751%
Argentina,2015,M,"Granite, porphyry, basalt, sandstone, other monumental and building stone, whether or not roughly trimmed, cut, by sawing etc, into blocks or slabs of a rectangular (including square) shape",2516,"$27,964",1,"$28,028",99.773%
Argentina,2016,M,Petroleum gases and other gaseous hydrocarbons,2711,"$10,851,739",1,"$10,980,451",98.828%
Argentina,2017,M,Petroleum gases and other gaseous hydrocarbons,2711,"$6,724,343",1,"$6,763,085",99.427%


In [37]:

partners_and_products['reporterDesc'].unique()

array(['Algeria', 'Brazil', 'Belgium', 'Cambodia', 'China', 'Croatia',
       'Canada', 'Chile', 'Congo', 'Denmark', 'France and Monaco',
       'Ecuador', 'Germany', 'China Hong Kong', 'Japan', 'Italy',
       'Republic of Korea', 'Mexico', 'Indonesia', 'Israel',
       "Côte d'Ivoire", 'Malaysia', 'Montenegro',
       'Asia, not elsewhere specified', 'Poland', 'Peru', 'Portugal',
       'Sao Tome and Principe', 'Morocco', 'Russian Federation',
       'Netherlands', 'Serbia', 'Slovenia', 'India', 'Singapore',
       'South Africa', 'Zimbabwe', 'Spain', 'Switzerland', 'Thailand',
       'United Arab Emirates', 'Former Sudan', 'Turkey', 'United Kingdom',
       'United States of America', 'Belize', 'Cameroon', 'Azerbaijan',
       'Botswana', 'Benin', 'Colombia', 'El Salvador', 'Guatemala',
       'Mauritius', 'Kazakhstan', 'Lebanon', 'Norway', 'Nicaragua',
       'Niger', 'Pakistan', 'Nigeria', 'Anguilla', 'Slovakia', 'Vietnam',
       'Sweden', 'Uruguay', 'Burkina Faso', 'Austria', 'B



Por cada combinação ano, país parceiro, produto importado, obtém as importações
desse produto pelo parceiro e calcula o ranking do país em análise nas importações
desse produto pelo parceiro.

In [38]:
combos_of_interest = partners_and_products[(partners_and_products['cmdCode'].isin(top_export_products_codes) 
                       & (partners_and_products['reporterCode'].isin(top_export_partners_codes)))][['reporterCode','cmdCode']].drop_duplicates()
combos_of_interest
combos_of_interest = [tuple(x) for x in combos_of_interest.values]
combos_of_interest

[(156, '2516'),
 (124, '2709'),
 (156, '2709'),
 (156, '7102'),
 (251, '2709'),
 (251, '2711'),
 (490, '2516'),
 (490, '2709'),
 (620, '2709'),
 (620, '7204'),
 (699, '2709'),
 (710, '2709'),
 (710, '2710'),
 (699, '7204'),
 (724, '0303'),
 (724, '2516'),
 (724, '2709'),
 (764, '2709'),
 (764, '7102'),
 (784, '7102'),
 (764, '7204'),
 (784, '7204'),
 (842, '2709'),
 (842, '2710'),
 (842, '2711'),
 (842, '7102'),
 (124, '2516'),
 (490, '7204'),
 (710, '7102'),
 (724, '2711'),
 (842, '9999'),
 (156, '2711'),
 (124, '7102'),
 (620, '2710'),
 (490, '2711'),
 (699, '2516'),
 (724, '7204'),
 (251, '9999'),
 (699, '7102'),
 (764, '2711'),
 (620, '2711'),
 (490, '9999'),
 (699, '2711'),
 (124, '9999'),
 (710, '2711'),
 (620, '2516'),
 (124, '2711'),
 (251, '2710'),
 (156, '2715')]

## Get the alternative suppliers for country_of_interest partners



In [41]:
cmdCode_list = [cmdCode for (countryCode,cmdCode) in combos_of_interest]
cmdCode_list = list(set(cmdCode_list))
countryCode_list = [countryCode for (countryCode,cmdCode) in combos_of_interest]
countryCode_list = list(set(countryCode_list))
cmdCodeCSV = ','.join(cmdCode_list)
countryCodeCSV = ','.join(map(str,countryCode_list))

In [42]:
print("Years of interest: ", years_of_interest)
print("countryCodeCSV: ", countryCodeCSV)
print("cmdCodeCSV: ", cmdCodeCSV)


df = comtrade.get_data(reporterCode=countryCodeCSV,
                       typeCode='C',  # commodities
                       freqCode='A',  # annual
                       partnerCode=None,
                       period=years_of_interest,
                       cmdCode=cmdCodeCSV,  # Top level product types
                       motCode=0,  # all means of transport
                       flowCode='M',
                       timeout=500)

Years of interest:  2007,2008,2009,2010,2014,2015,2016,2017,2018,2019,2020,2021
countryCodeCSV:  764,710,490,842,620,784,156,724,251,124,699
cmdCodeCSV:  2710,7204,2709,2516,7102,2711,9999,2715,0303


/Users/jrc/develop/cipf-comtrade/comtrade.py:344: UserWarning: Server returned HTTP Status: 500
  if days_since_modification <= 1:


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Alternative, concatenating

In [43]:
import pandas as pd

df2: pd.DataFrame = None
for countryCode,cmdCode in combos_of_interest:
    tmp = comtrade.get_data(reporterCode=countryCode,
                       typeCode='C',  # commodities
                       freqCode='A',  # annual
                       partnerCode=None,
                       period=years_of_interest,
                       cmdCode=cmdCode,  # Top level product types
                       motCode=0,  # all means of transport
                       flowCode='M',
                       timeout=500)
    if df2 is None:
        df2 = tmp
    else:
        df2 = pd.concat([df2,tmp],axis=0)
df=df2

/Users/jrc/develop/cipf-comtrade/comtrade.py:395: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  lenBefore = len(df)
/Users/jrc/develop/cipf-comtrade/comtrade.py:395: UserWarning: Query returned different isAggregate values, check for duplicate results when aggregating
  lenBefore = len(df)


In [ ]:
df.info()

In [137]:
import pandas as pd
        #  set max rows to 500
pd.set_option('display.max_rows', 1500)


other_suppliers = comtrade.total_rank_perc(df,
                                           groupby=['refYear','flowCode','reporterCode','cmdCode','partnerCode'],
                                           col='primaryValue',
                                           prefix='cmd_partner',
                                           )

other_suppliers = other_suppliers[other_suppliers[['reporterCode','cmdCode']].apply(tuple,axis=1).isin(combos_of_interest)]
cols=['reporterDesc','cmdCode','cmdDesc','refYear','cmd_partner_upper_sum','partnerDesc','cmd_partner_rank','cmd_partner_perc','primaryValue']
formats =  make_format(cols)

other_suppliers[(other_suppliers['partnerCode'] == country_of_interest)
               | (other_suppliers['cmd_partner_rank']<=10)  # show top 10 alternative supplier
                ][cols].sort_values(by=['reporterDesc','cmdCode','refYear','cmd_partner_rank']).head(250).style.format(formats).hide(axis="index")

reporterDesc,cmdCode,cmdDesc,refYear,cmd_partner_upper_sum,partnerDesc,cmd_partner_rank,cmd_partner_perc,primaryValue
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",India,1,12.329%,"$96,414,475"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",Japan,2,12.260%,"$95,873,220"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",Canada,3,9.594%,"$75,028,517"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",Egypt,4,7.158%,"$55,979,560"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",Brazil,5,6.842%,"$53,505,307"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",United States of America,6,6.622%,"$51,788,218"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",Italy,7,5.580%,"$43,637,504"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",Turkey,8,4.889%,"$38,235,486"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",Spain,9,4.660%,"$36,443,384"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",Russian Federation,10,3.356%,"$26,244,616"


In [131]:
other_suppliers[(other_suppliers['partnerCode'] == country_of_interest)][cols].sort_values(by=['reporterDesc','refYear','cmd_partner_upper_sum','cmd_partner_rank'],
                                                                                                ascending=[True,True,False,True]).head(500).style.format(formats).hide(axis='index')

/var/folders/bc/41rz3hqn5335_y1tbvy1w2_w0000gp/T/ipykernel_18070/2706241757.py:2: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  ascending=[True,True,False,True]).head(500).style.format(formats).hide_index()


reporterDesc,cmdCode,cmdDesc,refYear,cmd_partner_upper_sum,partnerDesc,cmd_partner_rank,cmd_partner_perc,primaryValue
China,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2001,"$17,516,515,040",Angola,9,4.119%,"$721,464,446"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2001,"$782,018,383",Angola,62,0.002%,"$16,967"
China,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2008,"$169,251,777,075",Angola,2,13.210%,"$22,358,967,995"
China,71,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin",2008,"$7,547,712,678",Angola,23,0.306%,"$23,098,324"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2008,"$6,116,497,799",Angola,66,0.007%,"$414,551"
China,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2011,"$275,766,336,712",Angola,2,8.997%,"$24,809,999,280"
China,71,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin",2011,"$14,911,409,892",Angola,19,0.666%,"$99,380,019"
China,25,"Salt; sulphur; earths, stone; plastering materials, lime and cement",2011,"$5,794,352,423",Angola,37,0.196%,"$11,337,407"
China,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",2014,"$316,787,667,435",Angola,2,9.805%,"$31,060,219,206"
China,71,"Natural, cultured pearls; precious, semi-precious stones; precious metals, metals clad with precious metal, and articles thereof; imitation jewellery; coin",2014,"$41,893,505,733",Angola,27,0.073%,"$30,495,543"


In [ ]:
other_suppliers.to_excel(f'{country_of_interest_desc}_export_partners_alternative.xlsx')

# AQUI

# Importações

## Principais produtos importados

In [ ]:
show_only_top=10
export_products = comtrade.top_commodities(country_of_interest,
                                               partnerCode=0,  # 0 = all partners
                                               years=years_of_interest, 
                                               flowCode='M',
                                           motCode=0,
                                           rank_filter=10)


In [ ]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdCode','cmdDesc', 'primaryValueFormated',
        'partnerDesc',
        comtrade.PERC_CMD_IN_PARTNER ]
export_products[cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdCode,cmdDesc,primaryValueFormated,partnerDesc,perc_cmd_for_partner
402,Angola,2021,Import,1,27,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes","1,886,757,547.56",World,16.58%
230,Angola,2021,Import,2,84,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","1,809,939,647.27",World,15.91%
126,Angola,2021,Import,3,87,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","779,123,345.04",World,6.85%
367,Angola,2021,Import,4,85,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","776,747,585.90",World,6.83%
161,Angola,2021,Import,5,10,Cereals,"584,957,170.07",World,5.14%
27,Angola,2021,Import,6,73,Iron or steel articles,"536,792,046.75",World,4.72%
362,Angola,2021,Import,7,39,Plastics and articles thereof,"405,040,625.98",World,3.56%
240,Angola,2021,Import,8,02,Meat and edible meat offal,"348,250,700.18",World,3.06%
322,Angola,2021,Import,9,30,Pharmaceutical products,"322,068,826.04",World,2.83%
169,Angola,2021,Import,10,15,"Animal, vegetable or microbial fats and oils and their cleavage products; prepared edible fats; animal or vegetable waxes","311,937,457.18",World,2.74%


## Principais produtos importados e origens

In [ ]:
show_only_top = 10
export_products = comtrade.top_commodities(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           flowCode='M',
                                           cmdCode='AG2',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [ ]:
cols = ['reporterDesc','refYear', 'flowDesc',	'rank_cmd', 'cmdDesc','rank_partner','partnerDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_products[comtrade.PERC_PARTNER_IN_CMD] > 0.05  # show only partners with more than 5% of trade
export_products[filter_relevant_partners][cols].style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_cmd,cmdDesc,rank_partner,partnerDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
3954,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",4,Togo,"687,679,076.53",100.00%,36.45%
13156,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",8,Netherlands,"318,111,982.79",73.62%,16.86%
13215,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",3,India,"291,213,622.52",39.05%,15.43%
3776,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",17,Malta,"184,307,230.08",99.00%,9.77%
4069,Angola,2021,Import,1,"Mineral fuels, mineral oils and products of their distillation; bituminous substances; mineral waxes",11,Belgium,"113,736,727.71",29.24%,6.03%
7421,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",1,China,"294,737,063.36",17.45%,16.28%
7026,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",2,Portugal,"219,815,557.96",16.19%,12.14%
1351,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",9,Italy,"206,406,850.29",48.33%,11.40%
7219,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",6,United Kingdom,"165,889,250.91",30.75%,9.17%
1111,Angola,2021,Import,2,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof",7,United States of America,"159,642,149.65",30.52%,8.82%


### Principal origem das importações

In [ ]:
show_ony_top=20
top_partners = comtrade.top_partners(country_of_interest,years_of_interest,
                                     flowCode='M', 
                                    rank_filter=show_ony_top)

In [ ]:
cols=['reporterDesc','refYear','flowDesc','rank_partner','partnerDesc','primaryValueFormated', comtrade.PERC_PARTNER_IN_CMD]
top_partners[cols].sort_values(by=['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,primaryValueFormated,perc_partner_for_cmd
258,Angola,2021,Import,1,China,"1,688,904,641.34",14.84%
95,Angola,2021,Import,2,Portugal,"1,357,979,633.71",11.93%
323,Angola,2021,Import,3,India,"745,723,527.22",6.55%
319,Angola,2021,Import,4,Togo,"687,689,058.08",6.04%
495,Angola,2021,Import,5,Brazil,"546,044,463.65",4.80%
610,Angola,2021,Import,6,United Kingdom,"539,546,476.65",4.74%
251,Angola,2021,Import,7,United States of America,"523,006,089.83",4.60%
455,Angola,2021,Import,8,Netherlands,"432,100,821.51",3.80%
192,Angola,2021,Import,9,Italy,"427,046,959.44",3.75%
36,Angola,2021,Import,10,France and Monaco,"411,360,213.52",3.62%


## Principais **parceiros** das importações e produtos importados

In [ ]:
show_only_top = 10
export_partners = comtrade.top_partners(country_of_interest,
                                           partnerCode=None,
                                           years=years_of_interest, 
                                           cmdCode='AG2',
                                           flowCode='M',
                                           motCode=0, 
                                           rank_filter=show_only_top)

In [ ]:
cols = ['reporterDesc','refYear', 'flowDesc','rank_partner','partnerDesc', 'cmdDesc', 'primaryValueFormated',
       comtrade.PERC_CMD_IN_PARTNER,comtrade.PERC_PARTNER_IN_CMD ]

filter_relevant_partners = export_partners[comtrade.PERC_PARTNER_IN_CMD] > 0.0  # show only partners with more than 5% of trade
export_partners[filter_relevant_partners][cols].sort_values(['refYear','flowDesc','rank_partner']).style.format({comtrade.PERC_CMD_IN_PARTNER: '{:,.2%}'.format, comtrade.PERC_PARTNER_IN_CMD: '{:,.2%}'.format})

,reporterDesc,refYear,flowDesc,rank_partner,partnerDesc,cmdDesc,primaryValueFormated,perc_cmd_for_partner,perc_partner_for_cmd
7421,Angola,2021,Import,1,China,"Machinery and mechanical appliances, boilers, nuclear reactors; parts thereof","294,737,063.36",17.45%,16.28%
1837,Angola,2021,Import,1,China,"Textiles, made up articles; sets; worn clothing and worn textile articles; rags","207,543,201.07",12.29%,80.04%
5764,Angola,2021,Import,1,China,"Electrical machinery and equipment and parts thereof; sound recorders and reproducers; television image and sound recorders and reproducers, parts and accessories of such articles","188,027,402.48",11.13%,24.21%
3683,Angola,2021,Import,1,China,"Vehicles; other than railway or tramway rolling stock, and parts and accessories thereof","170,241,136.72",10.08%,21.85%
12591,Angola,2021,Import,1,China,Iron and steel,"94,179,850.47",5.58%,64.83%
5371,Angola,2021,Import,1,China,Plastics and articles thereof,"90,509,512.27",5.36%,22.35%
11724,Angola,2021,Import,1,China,Glass and glassware,"62,248,517.42",3.69%,75.37%
6889,Angola,2021,Import,1,China,Iron or steel articles,"60,344,496.23",3.57%,11.24%
6008,Angola,2021,Import,1,China,Rubber and articles thereof,"58,498,228.59",3.46%,43.76%
11490,Angola,2021,Import,1,China,Pharmaceutical products,"48,373,928.61",2.86%,15.02%


## Análise de simetrias nos dados

In [ ]:


partnerA = country_of_interest


### Principais destinos de exportação reportados pelo país

In [ ]:
show_only_top = 30
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='X',
                     cmdCode='TOTAL',
                     reporterCode=partnerA,
                     partnerCode=None,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     customsCode=None,
                     echo_url=False
                     )
cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>100000][cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
348,Angola,China,World,2021,Export,"20,058,426,271.91",20058426271.912998,58.19%,All modes of transport,True,All customs procedure codes
211,Angola,India,World,2021,Export,"3,008,390,595.07",3008390595.072000,8.73%,All modes of transport,True,All customs procedure codes
288,Angola,United Arab Emirates,World,2021,Export,"1,399,449,578.20",1399449578.198000,4.06%,All modes of transport,True,All customs procedure codes
165,Angola,Thailand,World,2021,Export,"1,004,833,529.77",1004833529.770000,2.91%,All modes of transport,True,All customs procedure codes
46,Angola,Singapore,World,2021,Export,"802,659,408.76",802659408.761000,2.33%,All modes of transport,True,All customs procedure codes
109,Angola,Spain,World,2021,Export,"793,986,998.99",793986998.994000,2.30%,All modes of transport,True,All customs procedure codes
67,Angola,Italy,World,2021,Export,"632,025,040.54",632025040.539000,1.83%,All modes of transport,True,All customs procedure codes
279,Angola,South Africa,World,2021,Export,"519,358,721.33",519358721.333000,1.51%,All modes of transport,True,All customs procedure codes
70,Angola,Chile,World,2021,Export,"518,264,559.23",518264559.233000,1.50%,All modes of transport,True,All customs procedure codes
63,Angola,Netherlands,World,2021,Export,"477,332,019.79",477332019.785000,1.38%,All modes of transport,True,All customs procedure codes


### Principais destinos de exportação segundo o registo da importações do resto do mundo

In [ ]:
show_only_top=30
df_import_rep_a_world = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode='M',
                     cmdCode='TOTAL',
                     reporterCode=None,
                     partnerCode=partnerA,
                     partner2Code=0,
                     motCode = 0,
                     # customsCode=0,
                     period=years_of_interest,
                     echo_url=False
                     )
total_import = df_import_rep_a_world['primaryValue'].sum()
df_import_rep_a_world['perc'] = df_import_rep_a_world['primaryValue']/total_import
df_import_rep_a_world[df_import_rep_a_world['primaryValue']>1000][cols].sort_values(by=['refYear','primaryValue','reporterDesc',], ascending=[True,False,True]).head(show_only_top).style.format({'perc': '{:,.2%}'.format}) 

,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
29,China,Angola,World,2021,Import,"20,908,231,317.00",20908231317.000000,62.45%,All modes of transport,True,All customs procedure codes
324,India,Angola,World,2021,Import,"2,315,947,568.41",2315947568.408000,6.92%,All modes of transport,True,All customs procedure codes
388,United Arab Emirates,Angola,World,2021,Import,"1,685,701,656.52",1685701656.518000,5.04%,All modes of transport,True,All customs procedure codes
385,Thailand,Angola,World,2021,Import,"1,607,756,231.51",1607756231.510000,4.80%,All modes of transport,True,All customs procedure codes
413,United States of America,Angola,World,2021,Import,"1,089,250,402.00",1089250402.000000,3.25%,All modes of transport,True,All customs procedure codes
325,Singapore,Angola,World,2021,Import,"752,550,761.40",752550761.398000,2.25%,All modes of transport,True,All customs procedure codes
355,Spain,Angola,World,2021,Import,"739,484,782.77",739484782.768000,2.21%,All modes of transport,True,All customs procedure codes
5,Belgium,Angola,World,2021,Import,"615,024,126.87",615024126.871000,1.84%,All modes of transport,True,All customs procedure codes
28,Chile,Angola,World,2021,Import,"537,823,552.96",537823552.960000,1.61%,All modes of transport,True,All customs procedure codes
147,Indonesia,Angola,World,2021,Import,"463,466,072.00",463466072.000000,1.38%,All modes of transport,True,All customs procedure codes


## Análise de simetrias nos dados com um país específico

In [ ]:
partnerB = comtrade.COUNTRY_CODES_REVERSE['China']
flowCode = 'M'
flowDesc = comtrade.FLOWS_CODES[flowCode]


phrase = f"{comtrade.COUNTRY_CODES.get(partnerA)} {flowDesc} to {comtrade.COUNTRY_CODES.get(partnerB)} reported by {comtrade.COUNTRY_CODES.get(partnerA)}" \
 if flowCode == 'X' else f"{comtrade.COUNTRY_CODES.get(partnerA)} {flowDesc} from {comtrade.COUNTRY_CODES.get(partnerB)} reported by {comtrade.COUNTRY_CODES.get(partnerA)}"

cmd_detail = 'AG4'
show_only_top = 10

cols = ['reporterDesc','partnerDesc','partner2Desc','refYear','flowDesc','cmdDesc','primaryValueFormated','primaryValue','perc','motDesc','isAggregate','customsDesc'] 

In [ ]:
df_import_rep_a = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCode,
                     cmdCode=cmd_detail,
                     reporterCode=partnerA,
                     partnerCode=partnerB,
                     partner2Code=0,
                     period=2020,
                     motCode = 0,
                     echo_url=False
                     )


In [ ]:
if df_import_rep_a is not None:
    total_import = df_import_rep_a['primaryValue'].sum()
    df_import_rep_a['perc'] = df_import_rep_a['primaryValue']/total_import
    print(phrase)
    result = df_import_rep_a[cols].sort_values(by=['refYear','primaryValue'], ascending=[True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print("No",phrase)
    result = ''

result

Angola Import from China reported by Angola


,reporterDesc,partnerDesc,partner2Desc,refYear,flowDesc,cmdDesc,primaryValueFormated,primaryValue,perc,motDesc,isAggregate,customsDesc
1021,Angola,China,World,2020,Import,"Rail locomotives; (other than those of heading no. 8601), locomotive tenders","106,385,404.28",106385404.277000,7.42%,All modes of transport,True,All customs procedure codes
1382,Angola,China,World,2020,Import,"New pneumatic tyres, of rubber","50,669,463.76",50669463.757000,3.53%,All modes of transport,True,All customs procedure codes
1087,Angola,China,World,2020,Import,nan,"48,169,359.41",48169359.409000,3.36%,All modes of transport,True,All customs procedure codes
1722,Angola,China,World,2020,Import,"Iron or non-alloy steel; flat-rolled products, width 600mm or more, clad, plated or coated","40,716,106.16",40716106.155000,2.84%,All modes of transport,True,All customs procedure codes
508,Angola,China,World,2020,Import,"Electric transformers, static converters (e.g. rectifiers) and inductors","39,653,621.92",39653621.919000,2.76%,All modes of transport,True,All customs procedure codes
2340,Angola,China,World,2020,Import,"Iron or non-alloy steel; flat-rolled products of a width of 600mm or more, hot-rolled, not clad, plated or coated","39,392,570.45",39392570.454000,2.75%,All modes of transport,True,All customs procedure codes
381,Angola,China,World,2020,Import,"Telephone sets, including smartphones and other telephones for cellular/wireless networks; other apparatus for the transmission or reception of voice, images or other data (including wired/wireless networks), excluding items of 8443, 8525, 8527, or 8528","36,628,189.17",36628189.175000,2.55%,All modes of transport,True,All customs procedure codes
28,Angola,China,World,2020,Import,"Medicaments; (not goods of heading no. 3002, 3005 or 3006) consisting of mixed or unmixed products for therapeutic or prophylactic use, put up in measured doses (incl. those in the form of transdermal admin. systems) or packed for retail sale","29,087,289.30",29087289.296000,2.03%,All modes of transport,True,All customs procedure codes
2147,Angola,China,World,2020,Import,"Air conditioning machines; comprising a motor driven fan and elements for changing the temperature and humidity, including those machines in which the humidity cannot be separately regulated","22,047,690.54",22047690.539000,1.54%,All modes of transport,True,All customs procedure codes
2671,Angola,China,World,2020,Import,"Clock, watch and similar glasses, glasses for non-corrective or corrective spectacles, curved, bent, hallowed etc, not optically worked; hollow glass spheres and their segments for manufacture","21,833,158.76",21833158.762000,1.52%,All modes of transport,True,All customs procedure codes


In [ ]:
# invertemos

print(years_of_interest, partnerA, partnerB)

flowCodeInv = 'X' if flowCode == 'M' else 'M'
flowDesc = comtrade.FLOWS_CODES[flowCodeInv]


phrase = f"{comtrade.COUNTRY_CODES.get(partnerB)} {flowDesc} to {comtrade.COUNTRY_CODES.get(partnerA)} reported by {comtrade.COUNTRY_CODES.get(partnerB)}" if flowCodeInv == 'X'\
    else f"{comtrade.COUNTRY_CODES.get(partnerB)} {flowDesc} from {comtrade.COUNTRY_CODES.get(partnerA)} reported by {comtrade.COUNTRY_CODES.get(partnerB)}"


print(phrase)

df_import_rep_b = comtrade.get_data("C",# C for commodities, S for Services
                     "A",# (freqCode) A for annual and M for monthly
                     flowCode=flowCodeInv,
                     cmdCode=cmd_detail,
                     reporterCode=partnerB,
                     partnerCode=partnerA,
                     partner2Code=0,
                     period=years_of_interest,
                     motCode = 0,
                     echo_url=False
                     )
if df_import_rep_b is not None:
    total_import = df_import_rep_b['primaryValue'].sum()
    df_import_rep_b['perc'] = df_import_rep_b['primaryValue']/total_import
    result = df_import_rep_b[cols].sort_values(by=['refYear','reporterDesc','primaryValue'], ascending=[True,True,False]).head(show_only_top).style.format({'perc': '{:,.2%}'.format})
else:
    print("No",phrase)
    result = ''
result

2021 24 156
China Export to Angola reported by China


ReadTimeout: HTTPSConnectionPool(host='comtradeapi.un.org', port=443): Read timed out. (read timeout=10)